<a href="https://colab.research.google.com/github/reznimat/ppzd_sp/blob/main/semestralni_prace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Načtení knihoven**

In [27]:
import csv
import pandas as pd
import math as mt
import datetime as dt

**Načtení datových souborů**

In [28]:
# POZOR, pouze vzorek dat
#url_sales = 'https://raw.githubusercontent.com/reznimat/ppzd_sp/main/SAMPLE_Online_Sales.csv'
url_sales = 'https://raw.githubusercontent.com/reznimat/ppzd_sp/main/Online_Sales.csv'
df_sales = pd.read_csv(url_sales)

url_coupons = 'https://raw.githubusercontent.com/reznimat/ppzd_sp/main/Discount_Coupon.csv'
df_coupons = pd.read_csv(url_coupons,sep=";")

#dict_sales = df_sales.to_dict('records')

print('**Sales**')
print(df_sales.info())
print('\n------\n')
print('**Coupons**')
print(df_coupons.info())


**Sales**
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52924 entries, 0 to 52923
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CustomerID           52924 non-null  int64  
 1   Transaction_ID       52924 non-null  int64  
 2   Transaction_Date     52924 non-null  object 
 3   Product_SKU          52924 non-null  object 
 4   Product_Description  52924 non-null  object 
 5   Product_Category     52924 non-null  object 
 6   Quantity             52924 non-null  int64  
 7   Avg_Price            52924 non-null  float64
 8   Delivery_Charges     52924 non-null  float64
 9   Coupon_Status        52924 non-null  object 
dtypes: float64(2), int64(3), object(5)
memory usage: 4.0+ MB
None

------

**Coupons**
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----

**Obohacení df_sales o atribut Month (Date Abrv)**

In [29]:
# Obohacení df_sales o Month pole
for i, row in df_sales.iterrows():
  Month = pd.to_datetime(row.Transaction_Date).strftime("%b")
  df_sales["Month"] = Month

print('**Sales**')
print(df_sales.info())

**Sales**
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52924 entries, 0 to 52923
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CustomerID           52924 non-null  int64  
 1   Transaction_ID       52924 non-null  int64  
 2   Transaction_Date     52924 non-null  object 
 3   Product_SKU          52924 non-null  object 
 4   Product_Description  52924 non-null  object 
 5   Product_Category     52924 non-null  object 
 6   Quantity             52924 non-null  int64  
 7   Avg_Price            52924 non-null  float64
 8   Delivery_Charges     52924 non-null  float64
 9   Coupon_Status        52924 non-null  object 
 10  Month                52924 non-null  object 
dtypes: float64(2), int64(3), object(6)
memory usage: 4.4+ MB
None


**Spojení sales a coupons tabulek přes Product_Category a Month**

In [30]:
#Merge Sales a Coupons na základě Product Category a Month
df_sales_coupons = pd.merge(df_sales,df_coupons, on=["Product_Category","Month"])

print(df_sales_coupons.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52524 entries, 0 to 52523
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CustomerID           52524 non-null  int64  
 1   Transaction_ID       52524 non-null  int64  
 2   Transaction_Date     52524 non-null  object 
 3   Product_SKU          52524 non-null  object 
 4   Product_Description  52524 non-null  object 
 5   Product_Category     52524 non-null  object 
 6   Quantity             52524 non-null  int64  
 7   Avg_Price            52524 non-null  float64
 8   Delivery_Charges     52524 non-null  float64
 9   Coupon_Status        52524 non-null  object 
 10  Month                52524 non-null  object 
 11  Coupon_Code          52524 non-null  object 
 12  Discount_pct         52524 non-null  int64  
dtypes: float64(2), int64(4), object(7)
memory usage: 5.2+ MB
None


**Počet a výpis jedinečných produktových kategorií**

In [31]:
# Počet a výpis distinktních produktových kategorií
uniqPC = set()
for i, row in df_sales.iterrows():
  uniqPC.add(row["Product_Category"])

print("Produktových kategorií: "+ str(len(uniqPC)) )  # vypíše počet produktových kategorií
print(uniqPC)  # vypíše seznam produktových kategorií

Produktových kategorií: 20
{'Google', 'Office', 'Gift Cards', 'Bottles', 'Lifestyle', 'Accessories', 'Fun', 'Nest', 'Apparel', 'Notebooks & Journals', 'Bags', 'Drinkware', 'Housewares', 'Backpacks', 'Android', 'More Bags', 'Waze', 'Nest-Canada', 'Nest-USA', 'Headgear'}


**Suma tržeb pro kategorii Apparel**

In [32]:
# Hint: InvoiceValue = (( QuantityAvg_price)(1-Dicount_pct)*(1+GST))+Delivery_Charges
# nejsou započítány náklady na dopravu a daně

ApparelSales = 0

for i, row in df_sales_coupons.iterrows():
  if row.Product_Category == 'Apparel':
    if row.Coupon_Status =='Not Used':
      ApparelSales += (row.Quantity*row.Avg_Price) # Bez slevy
    else:
      ApparelSales += (row.Quantity*row.Avg_Price)*(1-(row.Discount_pct/100)) # Se slevou

print(ApparelSales)

442432.6169999865


In [34]:
uniqMnth = set()
for i, row in df_coupons.iterrows():
  uniqMnth.add(row["Month"])

print(uniqMnth)

{'Jan', 'Feb', 'May', 'Oct', 'Jun', 'Aug', 'Sep', 'Mar', 'Dec', 'Apr', 'Nov', 'Jul'}


**Tržby po kategoriích a měsících**

In [36]:
my_list = [] #prázdný list

for kategorie in uniqPC:
  for mesic in uniqMnth:
    CategorySales = 0
    for i, row in df_sales_coupons.iterrows():
      if row.Product_Category == kategorie and row.Month == mesic:
        if row.Coupon_Status =='Not Used':
          CategorySales += (row.Quantity*row.Avg_Price) # Bez slevy
        else:
          CategorySales += (row.Quantity*row.Avg_Price)*(1-(row.Discount_pct/100)) # Se slevou
    my_list.append((kategorie, mesic, CategorySales)) # append vypočítaný záznam do listu
#    print()
#    slovnik["kategorie"] = kategorie
#    slovnik["mesic"] = mesic
#    slovnik["Sales"] = CategorySales

df_CatMonSales = pd.DataFrame.from_records(my_list, columns=['kategorie', 'mesic','Sales']) # převod listu na data_frame

print(df_CatMonSales)

KeyboardInterrupt: 

In [40]:
print(df_CatMonSales)

    kategorie mesic    Sales
0      Google   Jan      0.0
1      Google   Feb      0.0
2      Google   May      0.0
3      Google   Oct      0.0
4      Google   Jun      0.0
..        ...   ...      ...
235  Headgear   Mar      0.0
236  Headgear   Dec  38873.2
237  Headgear   Apr      0.0
238  Headgear   Nov      0.0
239  Headgear   Jul      0.0

[240 rows x 3 columns]
